<hr style="color:green" />
<h1 style="color:green">COSC2673 Assignment 2: Image Classification for Cancerous Cells</h1>
<h2 style="color:green">File 01: Data Exploration</h2>
<hr style="color:green" />

<p>In this file, load the initial label files and do a basic examination of the contents</p>

In [1]:
# Import the required libraries
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

import data_basic_utility as dbutil
# import graphing_utility as graphutil
import statistics_utility as statsutil

# run this to have a randomised seed every time. For reproducable results, set this to a constant
random_seed = dbutil.get_random_seed()

c:\Users\nelso\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\nelso\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\nelso\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Load the files

In [2]:
# Use these locations for Google Colab, where we need the Github version of the file
# locationMain = "https://raw.githubusercontent.com/s2105802-Nelson/COSC2673-A2-Group42/main/Image_classification_data/data_labels_mainData.csv?token=GHSAT0AAAAAACBUMPCH5CK2I562VIUK4US4ZCB3SFQ"
# locationExtra = "https://raw.githubusercontent.com/s2105802-Nelson/COSC2673-A2-Group42/main/Image_classification_data/data_labels_extraData.csv?token=GHSAT0AAAAAACBUMPCHQZ5H2FWD4IEFU7WUZCB3SZA"

# Use these locations for Nelson's local version of the file
locationMain = "C:/Development/COSC2673A2/Image_classification_data/data_labels_mainData.csv"
locationExtra = "C:/Development/COSC2673A2/Image_classification_data/data_labels_extraData.csv"

dfMain = pd.read_csv(locationMain)
dfExtra = pd.read_csv(locationExtra)

dfMain.head()

,InstanceID,patientID,ImageName,cellTypeName,cellType,isCancerous
0,22405,1,22405.png,fibroblast,0,0
1,22406,1,22406.png,fibroblast,0,0
2,22407,1,22407.png,fibroblast,0,0
3,22408,1,22408.png,fibroblast,0,0
4,22409,1,22409.png,fibroblast,0,0


In [3]:
dfExtra.head()

,InstanceID,patientID,ImageName,isCancerous
0,12681,61,12681.png,0
1,12682,61,12682.png,0
2,12683,61,12683.png,0
3,12684,61,12684.png,0
4,12685,61,12685.png,0


First, examine the Main dataset. Review of the data, structure, types and formats. If there are any basic errors, fix or clean.

In [4]:
statsutil.df_basic_data_info(dfMain, "dfMain", largeTextWarning=False)

Dataframe Name: dfMain
Dataframe Shape (rows / columns): (9896, 6)
-----------------------------

dataframe.info()
--------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9896 entries, 0 to 9895
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   InstanceID    9896 non-null   int64 
 1   patientID     9896 non-null   int64 
 2   ImageName     9896 non-null   object
 3   cellTypeName  9896 non-null   object
 4   cellType      9896 non-null   int64 
 5   isCancerous   9896 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 464.0+ KB
None
-----------------------------

dataframe.describe()
--------------------
         InstanceID    patientID     cellType  isCancerous
count   9896.000000  9896.000000  9896.000000  9896.000000
mean   10193.880154    29.762025     1.501516     0.412187
std     6652.912660    17.486553     0.954867     0.492253
min        1.000000     1.000000     0.000000     0.0000

How many records per patient id?

In [5]:
dfMain.groupby("patientID").size()

patientID
1      19
2      33
3     136
4     127
5     169
6     198
7     253
8     332
9     348
10    302
11     56
12    130
13    180
14    207
15    125
16    111
17    310
18    320
19    158
20    325
21    224
22    152
23    254
24    192
25    180
26    157
27     17
28     15
29    355
30    110
31    137
32     99
33    163
34     14
35     11
36    128
37     71
38     84
39    105
40    209
41    250
42    136
43    137
44    121
45     74
46    120
47    133
48    147
49    187
50    195
51    286
52    178
53    132
54    389
55    263
56     92
57    149
58    161
59    115
60    115
dtype: int64

Patients have multiple records. Do patients have records that are both non-cancerous and cancerous?

In [10]:
dfCancerousGrouped = dfMain.groupby(["patientID", "isCancerous"]).size()
dfCancerousGrouped.head()

patientID  isCancerous
1          0               19
2          0               33
3          0              136
4          0              127
5          0              134
dtype: int64

In [11]:
# Now look for any patient id that has more than one record
dfCancerousGrouped.groupby("patientID").filter(lambda group: len(group) > 1).groupby("patientID").size()

patientID
5     2
6     2
7     2
8     2
9     2
10    2
13    2
14    2
17    2
18    2
19    2
20    2
21    2
22    2
30    2
31    2
32    2
36    2
38    2
40    2
41    2
42    2
43    2
46    2
47    2
48    2
49    2
50    2
51    2
52    2
53    2
54    2
55    2
60    2
dtype: int64

**Conclusion**

Patients have records for both cancerous and non-cancerous cells. Therefore, there is no need to split according to patient Id for Cancerous modelling, there won't be any data-leakage based on patient Id

Make the same check for Cell Type

In [12]:
dfCellTypeGrouped = dfMain.groupby(["patientID", "cellType"]).size()
dfCellTypeGrouped.head()

patientID  cellType
1          0           11
           1            6
           3            2
2          0            8
           1           18
dtype: int64

Already, without further querying, we can see that Patients can have multiple records for different cell types. Again, this means there is no need to split by patient id

Finally, check if there is any correlation between cell type and cancerous. If we group by both and do a count, we should see if there are any cells that only have cancerous or non cancerous, or whether any cell can be cancerous or not.

In [14]:
dfCtCancer = dfMain.groupby(["cellType", "isCancerous"]).size()
print(dfCtCancer.shape)
dfCtCancer.head(20)

(4,)


cellType  isCancerous
0         0              1888
1         0              2543
2         1              4079
3         0              1386
dtype: int64